In [1]:
import pandas as pd
import requests
import io

In [2]:
def download_file_from_google_drive(id):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    return get_response_content(response)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def get_response_content(response):
    CHUNK_SIZE = 32768

    result = None
    
    with io.BytesIO() as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
        
        result = f.getvalue()
    
    return result

In [3]:
file_bytes = download_file_from_google_drive('1S8tQHI1SPwdFPepDZu-muyVepGhlILzd')

In [4]:
freq_passwords = dict()

with io.BytesIO(file_bytes) as b:
    with io.TextIOWrapper(b, encoding='utf-8') as f:
        for line in f:
            if line:
                parts = line.split(',')
                freq_passwords[parts[0]] = int(parts[1])

In [5]:
test = pd.read_csv('Xtest.csv')

In [6]:
with open('output.csv', 'w+') as f:
    f.write('Id,Times\n')
    for index, row in test.iterrows():
        pswd = row['Password']
        count = 1
        
        if pswd in freq_passwords:
            count = freq_passwords[pswd]
        
        f.write('{},{}\n'.format(row['Id'], count))